In [ ]:
import os,sys
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sys
import os

# Add the project root directory to sys.path so Python can find src/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.config import conf

path = os.path.join(conf.data_path,'creditcard.csv')
df = pd.read_csv(path)

df['Hour'] = (df['Time'] // 3600) % 24
df['Day'] = df['Time'] // (3600 * 24)
df['LogAmount'] = np.log1p(df['Amount'])
fraud_per = df['Class'].mean() * 100


# print(f'Percentage of Frauds: {fraud_per:.2f}')
# print(f'Missing Values per Column:\n',df.isnull().sum())
# print(f'Statistical Summary of Data:\n',df.describe())
# print(f'DataFrame Information:\n')
# df.info()


# sns.histplot(data=df,x='LogAmount')
# plt.title('Histogram of LogAmount By Number of Transactions')
# plt.show()

# plt.figure(figsize=(12,6))
# sns.histplot(data=df[df['Class'] == 0],x='Hour',bins=24,color='green',kde=True,label='Legit')
# sns.histplot(data=df[df['Class'] == 1],x='Hour',bins=24,color='red',kde=True,label='Fraud')
# plt.title('Legit / Fraud Transactions By Hour')
# plt.legend()
# plt.grid()

plt.figure(figsize=(12,5))
sns.heatmap(data=df.corr(),annot=False)
plt.title('Correlational Heatmap of dataframe')

plt.show()

plt.figure(figsize=(12,5))
sns.boxplot(data=df,x='Class',y='Amount')
plt.yscale('log')
plt.title('Transaction Amount by Class')
plt.show()




X = df.drop(columns=['Time','Class'])
y = df['Class']

scaler = StandardScaler()
X_encoded = scaler.fit_transform(X)


pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_encoded)

df_pca = pd.DataFrame(data=X_pca,columns=['PCA1','PCA2'])
df_pca['Class'] = y.values

fig = px.scatter(df_pca,x='PCA1',y='PCA2',color='Class',title='PCA Projection (Fruad/Legit)')

if 'ipynbkernel' in sys.modules:
    fig.show()
else:
    fig.write_html(os.path.join(conf.reports_path,'pca_projection.html'))
    mlflow.log_artifact(os.path.join(conf.reports_path,'pca_projection.html'))
